In [2]:
import numpy as np
import pandas as pd

In [3]:
train_sc03 = pd.read_csv('train_sc03.csv')

In [4]:
test_sc03 = pd.read_csv('test_sc03.csv')

In [5]:
train_dataType = train_sc03.dtypes
train_dataType.value_counts()

float64    61093
int64        175
object         3
dtype: int64

In [6]:
def clean_data(data):
    uniq_id = data['SampleID']
    label = data['GA']
    data.drop(columns=['SampleID','data','randPerson'])
    x = data.drop(columns=['SampleID','data','randPerson', 'GA'])
    return x, label

In [7]:
x_train, y_train = clean_data(train_sc03)
x_test, y_test = clean_data(test_sc03)

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_X = scaler.fit_transform(x_train)
test_X = scaler.transform(x_test)

# Training ML Models

In [9]:
from sklearn.linear_model import LinearRegression

predictor = LinearRegression(n_jobs=-1)
predictor.fit(train_X, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [10]:
outcome = predictor.predict(test_X)
coefficients = predictor.coef_

In [11]:
from sklearn import metrics

In [12]:
def analysis(y_test, y_pred):
    print('MAE:',metrics.mean_absolute_error(y_test, y_pred))
    print('MSE:',metrics.mean_squared_error(y_test, y_pred))
    print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [13]:
analysis(y_test,outcome)

MAE: 1.4255988636937106e+16
MSE: 1.3608066510424544e+33
RMSE: 3.688911290668907e+16


In [14]:
outcome

array([-9.75455129e+15,  1.51073026e+15, -7.45384554e+00, -1.25926596e+17,
        3.95856149e+15, -1.77525064e+16, -2.87694627e+14,  5.30746434e+15,
       -2.16394350e+15, -2.99833910e+15, -1.41147641e+15,  1.39769476e+01])

In [15]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 500, random_state = 42)
# Train the model on training data
rf.fit(train_X, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [16]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_X)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 1.91


In [17]:
analysis(y_test,predictions)

MAE: 1.9075714285714263
MSE: 7.636214340136035
RMSE: 2.7633701055298463


In [18]:
submission = pd.DataFrame()
submission['SampleID'] = test_sc03['SampleID']
submission['GA'] = y_test
submission['predicted.GA'] = predictions
submission

,SampleID,GA,predicted.GA
0,I1,11.000000,12.767143
1,W1,11.000000,11.851429
2,B1,11.000000,11.364000
3,I2,15.000000,16.333143
4,W2,18.000000,15.843143
5,B2,16.000000,17.128857
6,I3,25.000000,25.304286
7,W3,27.000000,24.584286
8,B3,27.000000,23.987143
9,I4,2.571429,1.073429


In [19]:
submission.to_csv('GROUP-5--SC3--RandomForest.csv')